# Lab 2: Linear Systems Modeling

### Author: Yuriy Pasichnyk


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy import signal

In [ ]:
def print_signal(data, t, p_title='Signal plot', label=None, p_signal_axes='Voltage, V', extert_plt=False,
                        create_plt=True):
    # Plot
    if create_plt:
        plt.figure(figsize=(17, 7))
    
    plt.plot(t, data, label=label)
    
    if create_plt:
        plt.title(p_title)
        plt.xlabel('Time, s')
        plt.ylabel(p_signal_axes)
        plt.grid(True)
    
    if not extert_plt:
        if label is not None:
            plt.legend()
        plt.grid(True)
        plt.show()

def plot_signal_segment(signal, start_t, end_t, freq, p_title='Signal plot', label=None,
                        p_signal_axes='signal', p_size=(15, 7), get_segment=False, extert_plt=False,
                        create_plt=True):
    # Validate
    tatal_t = len(signal) / freq # time in sec
    if start_t > end_t or end_t > tatal_t:
        raise RuntimeError("Invalid time range boundaries!")
    
    # Data construct
    start_i = int(start_t * freq)
    end_i = int(end_t * freq)
    signal_s = signal[start_i:end_i]
    t = np.arange(start_t, end_t - 1 / freq, 1 / freq)
    
    # Plot
    if create_plt:
        plt.figure(figsize=p_size)
    
    plt.plot(t, signal_s, label=label)
    
    if create_plt:
        plt.title(p_title)
        plt.xlabel('Time, s')
        plt.ylabel(p_signal_axes)
    
    if not extert_plt:
        if label is not None:
            plt.legend()
        plt.grid(True)
        plt.show()
    
    if get_segment:
        return t, signal_s

# Моделювання роботи ЛДС з використанням різницевого рівняння.

## Task 1 

- В програмі задати вектори a та b коефіцієнтів рекурсивної та нерекурсивної частини ЛДС.

In [ ]:
def coeffs(d1, d2, m1, m2, p1, p2 , p3, p4):
    return (np.array([1, (d1 + d2) / 140, (p2 - d2) / 130, 0, - d1 / 150, (d1- m1) / 150]),  # a
            np.array([m1 / 10, (p3- d2) / 20, (m1 - m2) / 20, - p4 / 30, d2 / 20, - m2 / 20]))  # b

a, b = coeffs(1, 1, 0, 1, 2, 0, 0, 1)
print("a coeffs:", a)
print("b coeffs:", b)

## Task 2

#### Сформувати відліки синусоїдального сигналу:
- частоти 10 Гц
- тривалістю 1 сек
- амплітуди 1 В
- дискретизованого з частотою 256 Гц

#### Розрахувати реакцію системи на отриманий сигнал (функція lfilter) для двох випадків:
- 2.1. нульові початкові умови
- 2.2. випадкові початкові умови (скористатися функцією rand)

#### Для кожного:
- побудувати графіки вхідного та вихідного сигналів в одному вікні
- позначивши точки графіку, що відповідають відлікам, та огинаючі графіків.
- побудувати в окремому вікні перші 100 мс вхідного та вихідного сигналу.
- Зробити висновки щодо вигляду вихідного сигналу відносно вхідного (форма, амплітуда, спотворення, підсилення).

In [ ]:
sin_freq = 10  # Hz
sin_time = 1  # sec
sin_ampl = 1  # V
sin_s_freq = 256  # Hz

sin_t = np.arange(0., sin_time, 1. / sin_s_freq)
sin_s = np.sin(sin_t * 2 * np.pi * sin_freq)

print_signal(sin_s, sin_t, "Sinus Signal n=10Hz", sin_s_freq)

In [ ]:
cond_leng = max(len(a), len(b)) - 1
zero_cond = np.zeros(cond_leng)
rand_cond = np.random.rand(cond_leng)

sin_z_zero_cond, _ = signal.lfilter(b, a, sin_s, zi=zero_cond)
sin_z_rand_cond, _ = signal.lfilter(b, a, sin_s, zi=rand_cond)


print_signal(sin_s, sin_t, "LSM in/out for Sinus Signal with zero conditions",
            label="input", extert_plt=True)
print_signal(sin_z_zero_cond, sin_t,
            label="output", create_plt=False)

print_signal(sin_s, sin_t, "LSM in/out for Sinus Signal with random conditions",
            label="input", extert_plt=True)
print_signal(sin_z_rand_cond, sin_t,
            label="output", create_plt=False)

In [ ]:
plot_signal_segment(sin_s, 0, 0.1, sin_s_freq, "LSM in/out for Sinus Signal with zero conditions [fragment 0-100ms]",
            label="input", extert_plt=True)
plot_signal_segment(sin_z_zero_cond, 0, 0.1, sin_s_freq,
            label="output", create_plt=False)

plot_signal_segment(sin_s, 0, 0.1, sin_s_freq, "LSM in/out for Sinus Signal with random conditions [fragment 0-100ms]",
            label="input", extert_plt=True)
plot_signal_segment(sin_z_rand_cond, 0, 0.1, sin_s_freq,
            label="output", create_plt=False)

Щодо форма вихідного сигнала то вона далі залишилась синусоподібною та стала стиснутою до вісі часу, тобто амплітуда зменшилась, та відбувся зсув по фазі. На початку ми спостерігаємо спотворення, через установлення синусоїдальних коливань, яких небуло, і системі потрібен певний час, щоб стабілізуватись тому ми спостерігаємо спотворення. Спотворення з рандомними початковими даними є помітніші бо вони набувають більших відхилень від нуля, що прямопропорційно впливає на систему. Спотворення утворюються через те що не всі елементи різнецевого рівнняння вже нормально функціунують через початкові умови, які з часом витісняються тими, які задає сигнал. Ми отримуємо періодичну зміну підсилення на приглушення, яка при плавному стабільному сигналі має форму синусоїди. Система підсилює негативні значення сигналу (притягує їх до нуля) і послаблює позитивні значення сигналу, з відси висновок, що вона є у протифазі до нашого сигналу (візуально видно), але з певним зсувом.

## Task 3

Написати програму для визначення коефіцієнту передачі напруги ЛДС на частоті 10 Гц.

In [ ]:
def trasition_coef(inp, out):
    inp_x, inp_y = signal.find_peaks(inp, height=0)
    out_x, out_y = signal.find_peaks(out, height=0)
    inp_y, out_y = inp_y['peak_heights'], out_y['peak_heights']
    filt = inp_y != 0
    return np.average(out_y[filt] / inp_y[filt])

trasition_coef(sin_s, sin_z_zero_cond)

## Task 4

- Сформувати два синусоїдальних сигнали частоти 3 та 20 Гц тривалістю 1 с.
- Проілюструвати властивість адитивності системи, визначивши реакцію системи:
  - спочатку на кожний з сигналів окремо
  - на суму цих сигналів. 
- Проілюструвати властивість однорідності системи.
- Навести необхідні графіки.

In [ ]:
sin_s_3_20_freq = 256
sin_time_3_20 = 1

sin_t_3_20 = np.arange(0., sin_time_3_20, 1. / sin_s_3_20_freq)

sin_f_3 = 3
sin_s_3 = np.sin(sin_t_3_20 * 2 * np.pi * sin_f_3)

sin_f_20 = 20
sin_s_20 = np.sin(sin_t_3_20 * 2 * np.pi * sin_f_20)


sin_z3_zero_cond, _ = signal.lfilter(b, a, sin_s_3, zi=zero_cond)
sin_z20_zero_cond, _ = signal.lfilter(b, a, sin_s_20, zi=zero_cond)


print_signal(sin_s_3, sin_t_3_20, "LSM in/out for Sinus Signal n=3Hz with zero conditions",
            label="input", extert_plt=True)
print_signal(sin_z3_zero_cond, sin_t_3_20,
            label="output", create_plt=False)

print_signal(sin_s_20, sin_t_3_20, "LSM in/out for Sinus Signal n=20Hz with zero conditions",
            label="input", extert_plt=True)
print_signal(sin_z20_zero_cond, sin_t_3_20,
            label="output", create_plt=False)

In [ ]:
print_signal(sin_z20_zero_cond + sin_z3_zero_cond, sin_t_3_20, "LSM output for sum Sinus Signal n1=20Hz and n2=3Hz with zero conditions",
            label="separete influence", extert_plt=True)
print_signal(signal.lfilter(b, a, sin_s_3 + sin_s_20, zi=zero_cond)[0], sin_t_3_20,
            label="influence on the sum", create_plt=False)

### Summary: LSM satisfy the additivity property

In [ ]:
k_mult = 2
print_signal(sin_z3_zero_cond * k_mult, sin_t_3_20, "LSM output for sum Sinus Signal n1=20Hz and n2=3Hz with zero conditions",
            label="separete influence", extert_plt=True)
print_signal(signal.lfilter(b, a, sin_s_3 * k_mult , zi=zero_cond)[0], sin_t_3_20,
            label="influence on the sum", create_plt=False)

  ### Summary: LSM is Homogeous

## Task 5

- Розрахувати за допомогою функції lfilter перші 30 відліків імпульсної характеристики системи
    - вхід системи одиничний імпульс 
    - при нульових початкових умовах скориставшись для цього функцією unit_impulse
- Побудувати графіки вхідного та вихідного сигналу (функція stem).

In [ ]:
impuls_30 = signal.unit_impulse(30)
impuls_ch, _ = signal.lfilter(b, a, impuls_30, zi=zero_cond)

plt.figure(figsize=(17, 7))
plt.title("Impulse characteristic of LSM")
plt.xlabel('Time, s')
plt.ylabel("Amplitude, V")
plt.grid(True)

plt.stem(impuls_30, linefmt='grey', markerfmt='D', label="input")

plt.legend()
plt.show()

plt.figure(figsize=(17, 7))
plt.title("Impulse characteristic of LSM")
plt.xlabel('Time, s')
plt.ylabel("Amplitude, V")
plt.grid(True)

plt.stem(impuls_ch, label="output")

plt.legend()
plt.show()

## Task 6

- Створити систему з використанням коефіцієнтів чисельника і знаменника передавальної функції з використанням функції TransferFunction.
- Розрахувати 30 відліків імпульсної характеристики отриманої системи з використанням функції dimpulse.
- Порівняти результати з результатами __Task 5__
- побудувати графіки
- зробити висновки.

In [ ]:
transf_func = signal.TransferFunction(b, a, dt=1)
sys_t, sys_s =  signal.dimpulse(transf_func, n=30)
sys_s = np.squeeze(sys_s)
plt.figure(figsize=(17, 7))
plt.title("Impulse characteristic of LSM from TransferFunction")
plt.xlabel('Time, s')
plt.ylabel("Amplitude, V")
plt.grid(True)
plt.stem(sys_t, sys_s, label="output")
plt.show()

#### Summary:

Як і в п.5 ми отримали Імпульсну характеристику. Отже якщо підставити наші коефіцієнти з різнецевого рівняння і вставити в чисельник знаменик перехідної функції (TransferFunction) ми отримаєм одинкавий функціонал системи, який можна задати двома способами. Оскільки імпульсна характеристака унікально описує систему.

# Моделювання роботи ЛДС з використанням рівняння згортки.

## Task 7

- Розрахувати реакцію системи на сигнал з п. 2.1 з використанням функції розрахунку згортки convolve.
- Побудувати графіки вхідного та вихідного сигналу, аналогічні п. 2.1 (з нульовими початковими умовами).
- Всі отримані в п. 7 результати порівняти з п 2.1.
- Зробити висновки.

In [ ]:
sin_s_con = signal.convolve(sin_s, sys_s, 'same')

print_signal(sin_s, sin_t, "LSM input/output for Sinus Signal using convolution method with zero conditions",
            label="sin", extert_plt=True)
print_signal(sin_s_con, sin_t, label="conv", create_plt=False)

plot_signal_segment(sin_s, 0, 0.1, sin_s_freq, "LSM input/output for Sinus Signal using convolution method with zero conditionsinput/ [fragment 0-100ms]",
            label="input", extert_plt=True)
plot_signal_segment(sin_s_con, 0, 0.1, sin_s_freq, label="output", create_plt=False)

#### Summary:

За допомогою методу конволюції ми отрималитакий же результат (навіть кращий, щодо початкових умов), єдине що на кінці сигналу ми отримали пошкодження, яке спричинене нестачую наступних значень сигналу. Тобто в загальному використовуючи цей мето ми можем отримати кращі результати спочаку роботи системи і гіріші в кінці. 

# Моделювання роботи ЛДС в частотній області.

## Task 8

- Обчислити комплексну частотну характеристику системи з використанням функції freqz
- побудувати з її допомогою графіки АЧХ та ФЧХ
- Розрахувати 100 значень КЧХ для частоти дискретизації 256 Гц. 

In [ ]:
def rad_s2hz(rad_s_arr, freq):
    return freq * rad_s_arr / np.pi

def print_frequency_response_for_n(b, a, n, freq=None):  # КЧХ
    w, h = signal.freqz(b, a, worN=n)
    if freq is not None:
        w = rad_s2hz(w, freq)
    plt.figure(figsize=(15,5))
    plt.stem(w, np.real(h))
    if freq is None:
        plt.xlabel('Frequency (rad/sample)')
    else:
        plt.xlabel('Frequency (Hz)')
    plt.ylabel('Frequency response')
    plt.title(f'DSM frequency response for n={n}')
    plt.grid(True)
    plt.show()

def print_magniture_response(b, a, freq=None):  # АЧХ
    w, h = signal.freqz(b, a)
    if freq is not None:
        w = rad_s2hz(w, freq)
    plt.figure(figsize=(15,5))
    plt.plot(w, np.absolute(h))
    if freq is None:
        plt.xlabel('Frequency (rad/sample)')
    else:
        plt.xlabel('Frequency (Hz)')
    plt.ylabel('Amplitude Response')
    plt.title('DSM magniture response')
    plt.grid(True)
    plt.show()

def print_phase_response(b, a, freq=None):    # ФЧХ
    w, h = signal.freqz(b, a)
    if freq is not None:
        w = rad_s2hz(w, freq)
    phase = np.unwrap(np.arctan2(np.imag(h), np.real(h)))
    plt.figure(figsize=(15,5))
    plt.plot(w, phase)
    if freq is None:
        plt.xlabel('Frequency (rad/sample)')
    else:
        plt.xlabel('Frequency (Hz)')
    plt.ylabel('Phase (radians)')
    plt.title('DSM Phase Response')
    plt.grid(True)
    plt.show()
    


print_magniture_response(b, a)
print_phase_response(b, a)
print_frequency_response_for_n(b, a, 100)

## Task 9

- Визначити програмно проміжки частот, на яких система посилює сигнал (записати у висновках).
- Візуально пересвідчитись в правильності визначення коефіцієнта передачі і фазового зміщення, які розраховані в п. 3.
- Зробити висновки щодо величини коефіцієнта передачі системи на різних частотах.

In [ ]:
print_magniture_response(b, a, sin_s_freq)
trasition_coef(sin_s, sin_z_zero_cond)

По АЧХ чітко видно, що значення на всіх частотах менші за одиницю, що відповідно вказує на те, система завжди послаблює сигнал.

Візуально порівнявши функція з п.3 видає такийже же результат

На всіх частотах коефіцієнта передачі є меншим одиниці, що свідчить про те що наша система завжди понижує сигнал. Але вона ефективніше понижає такі складові частотного спектру сигналу: 50-80Hz та 75-210Hz

## Tasl 10 

- Побудувати функцію, яка визначає значення АЧХ та ФЧХ системи на довільній частоті.
- Перевірити за допомогою отриманої функції правильність розрахунків з п. 2.1, беручи до уваги, що в п.2.1 досліджувався синусоїдальний сигнал з частотою 10 Гц.
- Проілюструвати роботу функції для цього сигналу.

In [ ]:
def get_responses(b, a, signal_freq, samle_freq): # -> АЧХ, ФЧХ
    w, h = signal.freqz(b, a)
    w = rad_s2hz(w, samle_freq)
    
    phase = np.unwrap(np.arctan2(np.imag(h), np.real(h)))
    magni = np.absolute(h)
    
    for i, fr in enumerate(w):
        if fr > signal_freq:
            index = i - 1
            break;
    else:
        return None
    return magni[index], phase[index]
    
magn_resp, phase_resp = get_responses(b, a, 10, sin_s_freq)
print(f"magn_resp = {magn_resp}\nphase_resp = {phase_resp}")

## Task 11

- Розрахувати реакцію ЛДС на послідовність прямокутних імпульсів зі шпаруватістю 30%.
- Побудувати графіки вхідного та вихідного сигналів
- зробити висновки щодо спотворення вихідного сигналу відносно вхідного.

In [ ]:
duty_cicle = 0.7  # % active period
pwm_freq = 264

pwm_t = np.linspace(0, 10, pwm_freq * 10, endpoint=True)
pwm_s = signal.square(2 * np.pi * pwm_t, duty=duty_cicle)
pwm_z_zero_cond, _ = signal.lfilter(b, a, pwm_s, zi=zero_cond)

# Full signal Plot
print_signal(pwm_s, pwm_t, "LSM in/out for PWM duty=70% with random conditions",
            label="input", extert_plt=True)
print_signal(pwm_z_zero_cond, pwm_t, label="output", create_plt=False)

# Segment of signal Plot
plot_signal_segment(pwm_s, 0, 1.6, pwm_freq, "LSM in/out for PWM duty=70% with random conditions [fragment 0-1,6s]",
            label="input", extert_plt=True)
plot_signal_segment(pwm_z_zero_cond, 0, 1.6, pwm_freq, label="output", create_plt=False)

#### Summary:
З графіків можна прослідкувати, що система не встигає моментально відреагувати на митттєву зміну вхідного сигналу, і в таких місцях виникають спотворення, які видно на вихідному сигналі в місцях стрибків вхідного сигналу.